In [5]:
import pandas as pd
import numpy as np

In [6]:
def filter_stocks(kse100_df, stock_data):
    """
    Filters stocks based on the provided criteria.
    
    Parameters:
        kse100_df (pd.DataFrame): DataFrame of KSE-100 stocks.
        stock_data (pd.DataFrame): DataFrame containing stock details (Free Float Market Cap, daily turnover, etc.).
    Returns:
        pd.DataFrame: Filtered DataFrame of top 15 stocks based on Free Float Market Cap.
    """
    # Step 1: Filter stocks part of the KSE-100 Index
    kse100_stocks = stock_data[stock_data['Stock'].isin(kse100_df['Stock'])]

    # Step 2: Filter top 30 companies based on Free Float Market Capitalization
    top30_stocks = kse100_stocks.nlargest(30, 'FreeFloatMarketCap')

    # Step 3: Filter stocks with average daily turnover in the last year >= 100,000 shares
    top30_stocks['AverageDailyTurnover'] = top30_stocks['DailyTurnover'].apply(lambda x: sum(x) / len(x))
    filtered_by_turnover = top30_stocks[top30_stocks['AverageDailyTurnover'] >= 100_000]

    # Step 4: Filter stocks not traded below 10,000 shares on any single day
    def meets_minimum_trading_condition(daily_turnover, listing_days):
        return all(turnover >= 10_000 for turnover in daily_turnover[:min(len(daily_turnover), listing_days)])

    filtered_by_minimum_trading = filtered_by_turnover[
        filtered_by_turnover.apply(
            lambda row: meets_minimum_trading_condition(row['DailyTurnover'], row['ListingDays']), axis=1
        )
    ]

    # Step 5: Select the top 15 stocks based on Free Float Market Capitalization
    top15_stocks = filtered_by_minimum_trading.nlargest(15, 'FreeFloatMarketCap')

    return top15_stocks

# Adjusted DataFrame from the CSV file
kse100_df = pd.read_csv('daily_kse100_data.csv')
kse100_df = kse100_df.rename(columns={
    'Symbol': 'Stock',
    'Freefloat (M)': 'FreeFloatMarketCap',
    'Volume': 'DailyTurnover'
})
kse100_df['FreeFloatMarketCap'] = pd.to_numeric(kse100_df['FreeFloatMarketCap'].str.replace(',', ''), errors='coerce')
kse100_df['DailyTurnover'] = pd.to_numeric(kse100_df['DailyTurnover'].str.replace(',', ''), errors='coerce')
kse100_df['ListingDays'] = 365  # Placeholder value

# Mocking a 'stock_data' DataFrame for simulation
stock_data = kse100_df.copy()
stock_data['DailyTurnover'] = stock_data['DailyTurnover'].apply(
    lambda x: np.random.randint(50_000, 200_000, size=365).tolist()
)

# Apply the filter_stocks function
filtered_stocks = filter_stocks(kse100_df, stock_data)

# Display the results
print(filtered_stocks[['Stock', 'FreeFloatMarketCap', 'AverageDailyTurnover']])

     Stock  FreeFloatMarketCap  AverageDailyTurnover
1      KEL                2762         124592.484932
2   CNERGY                1648         123692.882192
0      BOP                1382         124665.208219
19    HUBC                 973         125219.235616
4     FCCL                 858         126625.841096
12   PIBTL                 804         123578.293151
34    BAHL                 778         124776.101370
37    BAFL                 710         125324.750685
24     FFC                 700         126193.841096
7      PPL                 668         124370.027397
8     OGDC                 645         121143.273973
47   EFERT                 601         126662.887671
3      PTC                 593         126475.263014
11     HBL                 587         127787.498630
42     DCR                 556         128692.884932


In [7]:
# def filter_stocks(kse100_df, stock_data):
#     """
#     Filters stocks based on the provided criteria.
    
#     Parameters:
#         kse100_df (pd.DataFrame): DataFrame of KSE-100 stocks.
#         stock_data (pd.DataFrame): DataFrame containing stock details (Free Float Market Cap, daily turnover, etc.).
        
#     Returns:
#         pd.DataFrame: Filtered DataFrame of top 15 stocks based on Free Float Market Cap.
#     """
#     # Step 1: Filter stocks part of the KSE-100 Index
#     kse100_stocks = stock_data[stock_data['Stock'].isin(kse100_df['Stock'])]

#     # Step 2: Filter top 30 companies based on Free Float Market Capitalization
#     top30_stocks = kse100_stocks.nlargest(30, 'FreeFloatMarketCap')

#     # Step 3: Filter stocks with average daily turnover in the last year >= 100,000 shares
#     top30_stocks['AverageDailyTurnover'] = top30_stocks['DailyTurnover'].apply(lambda x: sum(x) / len(x))
#     filtered_by_turnover = top30_stocks[top30_stocks['AverageDailyTurnover'] >= 100_000]

#     # Step 4: Filter stocks not traded below 10,000 shares on any single day
#     def meets_minimum_trading_condition(daily_turnover, listing_days):
#         return all(turnover >= 10_000 for turnover in daily_turnover[:min(len(daily_turnover), listing_days)])

#     filtered_by_minimum_trading = filtered_by_turnover[
#         filtered_by_turnover.apply(
#             lambda row: meets_minimum_trading_condition(row['DailyTurnover'], row['ListingDays']), axis=1
#         )
#     ]

#     # Step 5: Select the top 15 stocks based on Free Float Market Capitalization
#     top15_stocks = filtered_by_minimum_trading.nlargest(15, 'FreeFloatMarketCap')

#     return top15_stocks

# # Example usage
# # Load DF
# kse100_df = pd.read_csv('daily_kse100_data.csv')

# # kse100_df contains the list of stocks in the KSE-100 Index
# kse100_df = pd.DataFrame({'Stock': ['StockA', 'StockB', 'StockC', '...']})

# # Renaming relevant columns to match the function's expected format
# kse100_df = kse100_df.rename(columns={
#     'Symbol': 'Stock',
#     'Freefloat (M)': 'FreeFloatMarketCap',
#     'Volume': 'DailyTurnover'
# })

# # Converting relevant columns to numeric types for proper calculations
# kse100_df['FreeFloatMarketCap'] = pd.to_numeric(kse100_df['FreeFloatMarketCap'].str.replace(',', ''), errors='coerce')
# kse100_df['DailyTurnover'] = pd.to_numeric(kse100_df['DailyTurnover'].str.replace(',', ''), errors='coerce')

# # stock_data contains information for each stock
# # Each stock has a list of daily turnovers for the last year and the number of listing days
# stock_data = pd.DataFrame({
#     'Stock': ['StockA', 'StockB', 'StockC', 'StockD', '...'],
#     'FreeFloatMarketCap': [5000000, 3000000, 1000000, 2000000, '...'],
#     'DailyTurnover': [
#         [120000, 130000, 125000, 140000, '...'],  # Example turnovers for StockA
#         [90000, 110000, 100000, 95000, '...'],   # Example turnovers for StockB
#         [150000, 155000, 160000, 150000, '...'], # Example turnovers for StockC
#         [80000, 85000, 95000, 90000, '...'],     # Example turnovers for StockD
#     ],
#     'ListingDays': [365, 300, 365, 100, '...'],  # Example listing days for each stock
# })

# # Apply the filters
# filtered_stocks = filter_stocks(kse100_df, stock_data)

# # Display the result
# print(filtered_stocks)